In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline
import statsmodels.formula.api as smf

Populating the interactive namespace from numpy and matplotlib


# 1 Feature selection

Reducing complexity of the model (number of features or their dimensionality) is usually recommended in such cases. The most straighforward way of doing so is through feature selection. 

Select a subset of the regressors of the given size $k$ maximizing the model fit ($R^2$).  
Step-forward  
Step-backward  

Drawback - we're discarding information from the feature space.

In [2]:
n = 15
random.seed(10)

col = []
for i in range(n):
    col.append('x%d'%(i+1))

X = pd.DataFrame(np.random.randn(50*n).reshape(50,n))
Y = X.ix[:,:3].sum(axis = 1) + np.random.normal(0,1.1,50)
data = pd.concat((Y,X), axis = 1)
data.columns = ['Y'] + col
train = data[:20]
valid = data[20:]
formstr = 'Y ~ -1+'+' + '.join(col)
lm1 = smf.ols(formula=formstr, data = train).fit()
lmy1 = lm1.predict(valid)
R2v = 1 - ((lmy1-valid.Y)**2).sum() / ((valid.Y-valid.Y.mean())**2).sum()
#print('Validation R2 for 20 regressors is {0}'.format(R2v))

def regress(mask):
    formstr='Y~-1'
    for i in range(0,len(mask)):
        if mask[i]==1:
            formstr += '+x%d'%(i+1)
    lm2 = smf.ols(formula=formstr, data = train).fit()
#     lmy2 = lm2.predict(valid)
#     R2 = 1-sum(pow(lmy2-valid.Y,2))*(n-1)/sum(pow(valid.Y-mean(valid.Y),2))/(n-sum(mask)-1)
    return (lm2.rsquared_adj,lm2)#,(lm2.rsquared_adj == R2))   

mask0 = [0] * n  #initially exclude all regressors
r2_0 = 0
ix = []
while True:
    bestI = -1 #best regressor to exclude
    for i in range(0,n):
        if (mask0[i] == 0):
            mask = mask0[:]
            mask[i] = 1
            r2 = regress(mask)[0]
            #print((i,r2,r20))
            if (r2 > r2_0):
                r2_0 = r2
                bestI = i
    if (bestI >= 0):
        mask0[bestI] = 1
        ix.append(bestI+1)
        #print((bestI,r2_0))
        #print(regress(mask0)[-1])
    else:
        break
(r2,lm) = regress(mask0)
#print(lm.summary())
ix

[1, 4, 11, 3, 2, 14, 6]

In [7]:
#np.abs((data.corr() - np.eye(n+1))).max().max()

b = pd.DataFrame({'P-values':lm1.pvalues, 'Keeper':col})
c = 'greenyellow'
def highlight_pval(val):
    color = c if int(val[1:]) in ix else 'white'
    return 'background-color: %s'% color
# 'color: %s' % color
b.style.\
    applymap(highlight_pval, subset=['Keeper']).\
    bar(subset = ['P-values'], color=c)

In [5]:
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                     13.12
Date:                Mon, 26 Sep 2016   Prob (F-statistic):           5.62e-05
Time:                        09:30:58   Log-Likelihood:                -22.111
No. Observations:                  20   AIC:                             58.22
Df Residuals:                      13   BIC:                             65.19
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1             1.5347      0.222      6.925      0.0

In [6]:
mask0

[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0]

In [8]:
print(lm1.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.647
Method:                 Least Squares   F-statistic:                     3.445
Date:                Mon, 26 Sep 2016   Prob (F-statistic):             0.0890
Time:                        09:31:35   Log-Likelihood:                -18.710
No. Observations:                  20   AIC:                             67.42
Df Residuals:                       5   BIC:                             82.36
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1             2.1863      0.616      3.550      0.0